# Pipeline to manage anomalies processing

Before using any dataset, it is necessary to clean the data to obtain a clear, formatted, and complete dataset. This tedious step is the first step in data analysis.

In this notebook, there are building blocks (functions) that can be used later to perform the identified preprocessing steps.

- Remove missing values
- Remove duplicates
- Remove special characters
- Convert numbers to letters
- Identify the language of the review and translate it if necessary
- Correct spelling errors

In [2]:
import polars as pl
import re
from num2words import num2words
import langid
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from deep_translator import GoogleTranslator
from spellchecker import SpellChecker

## Remove missing values

In [3]:
def clean_missing_values(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
    """
    Remove rows from a DataFrame where the specified column has missing values,
    and return the cleaned DataFrame.
    
    Args:
        df (pl.DataFrame): The DataFrame to clean.
        column_name (str): Column to check for missing values.
        
    Returns:
        pl.DataFrame: Cleaned DataFrame.
    """
    # Drop rows where the specified column is null
    df_clean = df.drop_nulls(subset=[column_name])
    
    return df_clean

In [4]:
# Example of use
df = pl.read_csv("../data/processed/all_reviews.csv")
df_clean_missing_values = clean_missing_values(df, "review")
print(df.shape)
print(df_clean_missing_values.shape)

(3363667, 5)
(2835129, 5)


## Remove duplicates

In [4]:
def remove_duplicates(df: pl.DataFrame, subset_columns: list) -> pl.DataFrame:
    """
    Remove duplicate rows from a DataFrame based on specified columns,
    and return the cleaned DataFrame.
    
    Args:
        df (pl.DataFrame): The DataFrame to clean.
        subset_columns (list): List of columns to consider for duplicates.
        
    Returns:
        pl.DataFrame: DataFrame with duplicates removed.
    """
    # Drop duplicates based on the subset of columns
    df_clean = df.unique(subset=subset_columns)
    
    return df_clean

In [5]:
# Example of use
df_clean_duplicates = remove_duplicates(df_clean_missing_values, "review")
print(df_clean_missing_values.shape)
print(df_clean_duplicates.shape)

(2835129, 5)
(2811720, 5)


## Remove spacial characters

In [6]:
def remove_special_characters(df: pl.DataFrame, column_name: str, keep: str = "") -> pl.DataFrame:
    """
    Remove special characters from a specified text column using regex.

    Args:
        df (pl.DataFrame): Input Polars DataFrame.
        column_name (str): Name of the text column to clean.
        keep (str): Optional string of characters to preserve (e.g., ".," to keep dots and commas).

    Returns:
        pl.DataFrame: New DataFrame with cleaned text in the specified column.
    """
    # Build regex dynamically: allow alphanumeric, space, underscore, and chosen extra characters
    pattern = rf"[^\w\s{re.escape(keep)}]"

    def clean_text(text: str) -> str:
        if not isinstance(text, str) or not text.strip():
            return text  # ignore empty or non-string
        return re.sub(pattern, "", text)

    df_cleaned = df.with_columns(
        pl.col(column_name).map_elements(clean_text).alias(column_name)
    )

    return df_cleaned

In [7]:
# Example of use
df_no_special_character = remove_special_characters(df_clean_duplicates, "review", ".,!?")
print(df_no_special_character.head())

shape: (5, 5)
┌───────────┬────────────────────────────┬───────────────────────────┬─────────────┬───────────────┐
│ id_review ┆ review                     ┆ original_dataset          ┆ original_id ┆ service_type  │
│ ---       ┆ ---                        ┆ ---                       ┆ ---         ┆ ---           │
│ i64       ┆ str                        ┆ str                       ┆ i64         ┆ str           │
╞═══════════╪════════════════════════════╪═══════════════════════════╪═════════════╪═══════════════╡
│ 238627    ┆ I have stayed at this      ┆ data_tripadvisor_hotel_re ┆ 11579655    ┆ accommodation │
│           ┆ hotel at…                  ┆ views                     ┆             ┆               │
│ 3063439   ┆ The booking of rooms being ┆ data_european_hotel_revie ┆ 215511      ┆ accommodation │
│           ┆ no…                        ┆ ws                        ┆             ┆               │
│ 2505326   ┆ We chose this hotel        ┆ data_booking              ┆ 149017

## Convert numbers to letters

In [8]:
def numbers_to_words(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
    """
    Convert all numbers in a text column into words using num2words.

    Args:
        df (pl.DataFrame): Input DataFrame.
        column_name (str): Name of the text column to process.
        lang (str): Language code (e.g., 'en' or 'fr').

    Returns:
        pl.DataFrame: New DataFrame with numbers replaced by words.
    """
    def convert_numbers(text: str) -> str:
        if not isinstance(text, str) or not text.strip():
            return text
        # Replace every number with its text version
        return re.sub(r'\b\d+\b', lambda m: num2words(int(m.group())), text)

    df_converted = df.with_columns(
        pl.col(column_name).map_elements(convert_numbers).alias(column_name)
    )

    return df_converted

In [9]:
# Example of use
df_without_number = numbers_to_words(df_no_special_character, "review")
print(df_without_number.head())

shape: (5, 5)
┌───────────┬────────────────────────────┬───────────────────────────┬─────────────┬───────────────┐
│ id_review ┆ review                     ┆ original_dataset          ┆ original_id ┆ service_type  │
│ ---       ┆ ---                        ┆ ---                       ┆ ---         ┆ ---           │
│ i64       ┆ str                        ┆ str                       ┆ i64         ┆ str           │
╞═══════════╪════════════════════════════╪═══════════════════════════╪═════════════╪═══════════════╡
│ 238627    ┆ I have stayed at this      ┆ data_tripadvisor_hotel_re ┆ 11579655    ┆ accommodation │
│           ┆ hotel at…                  ┆ views                     ┆             ┆               │
│ 3063439   ┆ The booking of rooms being ┆ data_european_hotel_revie ┆ 215511      ┆ accommodation │
│           ┆ no…                        ┆ ws                        ┆             ┆               │
│ 2505326   ┆ We chose this hotel        ┆ data_booking              ┆ 149017

## Languages and translation

### Languages detection

In [10]:
def detect_language_parallel(df: pl.DataFrame, column_name: str, num_threads: int = 4) -> pl.DataFrame:
    """
    Detect the language of a text column in a Polars DataFrame using langid in parallel.

    Args:
        df (pl.DataFrame): Input DataFrame.
        column_name (str): Name of the text column to process.
        num_threads (int): Number of threads to use for parallel processing (default=4).

    Returns:
        pl.DataFrame: New DataFrame with an added column 'detected_lang' containing language codes.
    """
    def detect_lang(text: str) -> str:
        """Return the language code of a single text using langid."""
        if not isinstance(text, str) or not text.strip():
            return None
        lang, _ = langid.classify(text)
        return lang

    # Convert the Polars column to a Python list
    texts = df[column_name].to_list()

    # Parallel processing with ThreadPoolExecutor
    all_langs = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for result in tqdm(executor.map(detect_lang, texts), total=len(texts), desc="Language detection"):
            all_langs.append(result)

    # Return new DataFrame with added column
    df_result = df.with_columns(pl.Series("detected_lang", all_langs))
    return df_result

In [11]:
# Example of use
df_new = detect_language_parallel(df.head(100), column_name="review", num_threads=4)
counts = df_new.group_by("detected_lang").agg(pl.len().alias("nb_texts")).sort("nb_texts", descending=True)
print(counts)

Language detection: 100%|██████████| 100/100 [00:05<00:00, 19.10it/s]

shape: (4, 2)
┌───────────────┬──────────┐
│ detected_lang ┆ nb_texts │
│ ---           ┆ ---      │
│ str           ┆ u32      │
╞═══════════════╪══════════╡
│ en            ┆ 97       │
│ pl            ┆ 1        │
│ sv            ┆ 1        │
│ it            ┆ 1        │
└───────────────┴──────────┘


In [12]:
non_english_reviews = df_new.filter(pl.col("detected_lang") != "en")
print(non_english_reviews)

shape: (3, 6)
┌───────────┬────────────────────┬────────────────────┬─────────────┬──────────────┬───────────────┐
│ id_review ┆ review             ┆ original_dataset   ┆ original_id ┆ service_type ┆ detected_lang │
│ ---       ┆ ---                ┆ ---                ┆ ---         ┆ ---          ┆ ---           │
│ i64       ┆ str                ┆ str                ┆ i64         ┆ str          ┆ str           │
╞═══════════╪════════════════════╪════════════════════╪═════════════╪══════════════╪═══════════════╡
│ 61        ┆ Relaxing,          ┆ data_activities_re ┆ 62          ┆ leisure      ┆ it            │
│           ┆ Swimming,          ┆ views              ┆             ┆              ┆               │
│           ┆ vacationin…        ┆                    ┆             ┆              ┆               │
│ 77        ┆ Bingo   Space   TV ┆ data_activities_re ┆ 78          ┆ leisure      ┆ pl            │
│           ┆                    ┆ views              ┆             ┆        

### Translation in english

In [13]:
def translate_non_english_parallel(df: pl.DataFrame, column_name: str, detected_lang_col: str = "detected_lang", num_threads: int = 4) -> pl.DataFrame:
    """
    Translate texts in a Polars DataFrame column to English in parallel, only for texts detected in a language other than English.

    Args:
        df (pl.DataFrame): Input DataFrame.
        column_name (str): Name of the text column to translate.
        detected_lang_col (str): Name of the column containing detected language codes (default='detected_lang').
        num_threads (int): Number of threads to use for parallel translation (default=4).

    Returns:
        pl.DataFrame: New DataFrame with an added column 'translated_text' containing English translations.
                      Texts already in English are kept unchanged.
    """
    # Initialize the translator
    translator = GoogleTranslator(source='auto', target='en')

    def translate_one(text: str) -> str:
        """Translate a single text to English, return the text unchanged if empty or already in English."""
        if not isinstance(text, str) or not text.strip():
            return text
        try:
            return translator.translate(text)
        except Exception as e:
            return f"[ERROR: {e}]"

    # Prepare texts to translate
    texts_to_translate = []
    indices_to_translate = []
    for i, (text, lang) in enumerate(zip(df[column_name].to_list(), df[detected_lang_col].to_list())):
        if lang != 'en':
            texts_to_translate.append(text)
            indices_to_translate.append(i)

    # Parallel translation
    translated_texts = [None] * df.height
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for idx, result in zip(indices_to_translate, tqdm(executor.map(translate_one, texts_to_translate), total=len(texts_to_translate), desc="Translating non-English")):
            translated_texts[idx] = result

    # Fill in English texts unchanged
    for i in range(df.height):
        if translated_texts[i] is None:
            translated_texts[i] = df[column_name][i]

    # Return new DataFrame with added column
    df_result = df.with_columns(pl.Series("translated_text", translated_texts))
    return df_result


In [14]:
# Example of use 
df_translated = translate_non_english_parallel(df_new, column_name="review", detected_lang_col="detected_lang", num_threads=4)
print(df_translated)

Translating non-English:  67%|██████▋   | 2/3 [00:01<00:00,  1.97it/s]

shape: (100, 7)
┌───────────┬──────────────┬──────────────┬─────────────┬──────────────┬─────────────┬─────────────┐
│ id_review ┆ review       ┆ original_dat ┆ original_id ┆ service_type ┆ detected_la ┆ translated_ │
│ ---       ┆ ---          ┆ aset         ┆ ---         ┆ ---          ┆ ng          ┆ text        │
│ i64       ┆ str          ┆ ---          ┆ i64         ┆ str          ┆ ---         ┆ ---         │
│           ┆              ┆ str          ┆             ┆              ┆ str         ┆ str         │
╞═══════════╪══════════════╪══════════════╪═════════════╪══════════════╪═════════════╪═════════════╡
│ 0         ┆ fishing ,    ┆ data_activit ┆ 1           ┆ leisure      ┆ en          ┆ fishing ,   │
│           ┆ swimming ,   ┆ ies_reviews  ┆             ┆              ┆             ┆ swimming ,  │
│           ┆ traveling    ┆              ┆             ┆              ┆             ┆ traveling   │
│ 1         ┆ Snowball     ┆ data_activit ┆ 2           ┆ leisure      ┆ en

## Correction of spelling error

### Tests/ Expertimentation

Two methods with two libraries are available. Spellchecker seems to perform slightly better.

In [15]:
from textblob import TextBlob

def textblob_correct(text):
    blob = TextBlob(text)
    return str(blob.correct())

print(textblob_correct("I liek eat appels"))

I like eat appeals


In [16]:
from spellchecker import SpellChecker

spelling = SpellChecker()

def spelling_checks(text):
    correct_result = []
    typo_words = spelling.unknown(text.split())
    for word in text.split():
        if word in typo_words:
            correct_result.append(spelling.correction(word))
        else:
            correct_result.append(word)
    return " ".join(correct_result)

print(spelling_checks("I liek eat appels"))

I like eat apples


### Generalization

In [17]:
def correct_spelling_optimized(df: pl.DataFrame, column_name: str, batch_size: int = 10000) -> pl.DataFrame:
    """
    Apply fast spell correction on a text column of a large Polars DataFrame.
    Uses caching and batch processing for performance.

    Args:
        df (pl.DataFrame): Input Polars DataFrame.
        column_name (str): Column containing text to correct.
        batch_size (int): Number of rows to process per batch (default=10,000).

    Returns:
        pl.DataFrame: DataFrame with corrected text in the specified column.
    """
    spell = SpellChecker()
    cache = {}

    def fix_word(word: str) -> str:
        """Return cached or corrected version of a single word."""
        if word in cache:
            return cache[word]
        if word in spell:
            cache[word] = word
        else:
            corrected = spell.correction(word)
            cache[word] = corrected if corrected is not None else word
        return cache[word]

    def fix_text(text: str) -> str:
        """Apply correction to an entire review."""
        if not isinstance(text, str) or not text.strip():
            return text
        words = text.split()
        return " ".join(fix_word(w) for w in words)

    # Batch processing
    corrected_reviews = []
    num_rows = len(df)

    for i in tqdm(range(0, num_rows, batch_size), desc="Spell-checking in batches"):
        batch = df.slice(i, batch_size)
        corrected_batch = [fix_text(t) for t in batch[column_name]]
        corrected_reviews.extend(corrected_batch)

    # Return new DataFrame with corrected text
    df_corrected = df.with_columns(
        pl.Series(name=column_name, values=corrected_reviews)
    )

    return df_corrected


In [18]:
# Example of use
df_correct_spelling = correct_spelling_optimized(df_without_number.head(100), "review")
print(df_correct_spelling.head())

Spell-checking in batches: 100%|██████████| 1/1 [01:50<00:00, 110.48s/it]

shape: (5, 5)
┌───────────┬────────────────────────┬───────────────────────────────┬─────────────┬───────────────┐
│ id_review ┆ review                 ┆ original_dataset              ┆ original_id ┆ service_type  │
│ ---       ┆ ---                    ┆ ---                           ┆ ---         ┆ ---           │
│ i64       ┆ str                    ┆ str                           ┆ i64         ┆ str           │
╞═══════════╪════════════════════════╪═══════════════════════════════╪═════════════╪═══════════════╡
│ 238627    ┆ I have stayed at this  ┆ data_tripadvisor_hotel_review ┆ 11579655    ┆ accommodation │
│           ┆ hotel at…              ┆ s                             ┆             ┆               │
│ 3063439   ┆ The booking of rooms   ┆ data_european_hotel_reviews   ┆ 215511      ┆ accommodation │
│           ┆ being non…             ┆                               ┆             ┆               │
│ 2505326   ┆ We chose this hotel    ┆ data_booking                  ┆ 149017

## Number of reviews per dataset

In [19]:
df = df_without_number
result = df.group_by("original_dataset").agg(pl.len().alias("nb_lignes"))

print(df.shape)
print (result.to_pandas())
print(df.dtypes)

(2811720, 5)
                    original_dataset  nb_lignes
0               data_hotel_reviews_1      34393
1            data_activities_reviews      45874
2   data_european_restaurant_reviews       1426
3               data_hotel_reviews_2       9906
4             data_airline_reviews_2       8099
5        data_european_hotel_reviews     499288
6          data_restaurant_reviews_2       9330
7                       data_booking    1301650
8          data_restaurant_reviews_1        996
9               data_hotel_reviews_3       9653
10            data_airline_reviews_1       3692
11    data_tripadvisor_hotel_reviews     877413
12                      data_twitter      10000
[Int64, String, String, Int64, String]
